In [6]:
#!pip install urlib2

  Could not find a version that satisfies the requirement urlib2 (from versions: )
No matching distribution found for urlib2
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [282]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import requests
import urllib.request
import time
from sys import stdout
from time import sleep

### STEP 1: GET NEXT-PAGE-CURSOR AND ALL LEASE URL

In [ ]:
"""
Characteristics of this website: 1/ Infinite scroll down with no next page button 2/Log in Required to download files
get_url function is to web scrap each next-page url and corresponding lease doc url
paramters: page - starting page to web scrap
           cursor_prefix - the prefix of each page url
           page_limite - limit of pages to scroll down
           
key variable: url- a dictionary with lease doc url as keys, and its unique suffix as values
output: page_url - a dictionary with page url as key, and corresponding url dictionary as values

"""

def get_url(page,curor_prefix, page_limit=100):
    
    page_url=dict()
    count=0
    #page = 'https://www.lawinsider.com/contracts/tagged/lease-agreement'
    #cursor_prefix='https://www.lawinsider.com/contracts/tagged/lease-agreement?cursor='


    while (len(page)>0) and (len(page_url.keys())<=page_limit):
       
            count+=1
            url=dict()
            r = requests.get(page)
            soup = BeautifulSoup(r.text, "lxml")
            for sub in soup.find(id='pagination-append').find_all(href=re.compile(r'/contracts/(.*)')):
                url['https://www.lawinsider.com{}'.format(sub['href'])]=re.compile(r'(/contracts/([\d\w]+))/(.*)').search(sub['href']).group(3).replace('/','_')

            page_url[page]=url
            stdout.write('\r{} pages to scroll down'.format(str(page_limit+1-count)))
            stdout.flush()
            sleep(1)
            page=cursor_prefix+soup.find(id='pagination-append').find('a').attrs['data-next-cursor']
        #except:
            #print('ERROR')
            #continue
        
    return page_url
  

In [303]:
##TEST CASE

url_dict=get_url('https://www.lawinsider.com/contracts/tagged/lease-agreement',
       'https://www.lawinsider.com/contracts/tagged/lease-agreement?cursor=',
       page_limit=2)

0 pages to scroll down

### STEP 2: WEB SCRAP URLs Using SELENIUM

In [305]:
#!pip install selenium
"""
Need to download the webdriver which is webbrowser-specific 
For chrome: http://chromedriver.chromium.org/downloads

"""

#### SELENIUM LOGIN AND SET THE FILE DOWNLOAD DIRECTORY

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

dl_dir = "/Users/yanbincen/Desktop/missionpossible2019/sample_data_download"
chrome_path = "/Users/yanbincen/anaconda/lib/python3.6/site-packages/chromedriver"
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : dl_dir}
chromeOptions.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chromeOptions)

#### CLICK LOG IN PAGE AND FILL OUT LIG ON INFO

In [ ]:
log_in_page = 'https://accounts.google.com/signin/oauth/identifier?client_id=948740859141-jj4vlg2tqi6fqj62q7mvbo8jjam6dclv.apps.googleusercontent.com&as=Onvu0NDyehK7gm0A8Rd1kA&destination=https%3A%2F%2Fwww.lawinsider.com&approval_state=!ChR6SE1fSjhUTG5pbjQwd2ZYUVZ4QhIfNDZ2cnNraGQyZWNYQU9FaERQUmpBazJxT19UYmhCWQ∙APNbktkAAAAAXD4ufnraS8L9ERp8GB3Vj80bukBBQMeA&oauthgdpr=1&oauthriskyscope=1&xsrfsig=ChkAeAh8T_pYE_tn-OJaCsz09ZIo9DWoUW-tEg5hcHByb3ZhbF9zdGF0ZRILZGVzdGluYXRpb24SBXNvYWN1Eg9vYXV0aHJpc2t5c2NvcGU&flowName=GeneralOAuthFlow'
email = 'missionpossible2019nyc@gmail.com'
password = 'missionimpossible1999'
driver.get(log_in_page)
time.sleep(5)

driver.find_element_by_id("identifierId").send_keys(email)
driver.find_element_by_id("identifierNext").click()
time.sleep(5)
driver.find_element_by_name("password").send_keys(password)
element = driver.find_element_by_id('passwordNext')
driver.execute_script("arguments[0].click();", element)

#### MULTIPLE DOWNLOAD TEST CASE; DAILY LIMIT 50

In [416]:
test_url=[k for ud in url_dict.values() for k in ud.keys()]
p=re.compile(r'(https://www.lawinsider.com/contracts/([\S]+))/([\S-]+)/\d+/([\S]+)')

for url in test_url:
   
    company_name = p.search(url).group(3)
    date = p.search(url).group(4)
    doc_url=p.search(url).group(1)+'.docx'
    driver.get(doc_url)
    print('{} left; {} done'.format(len(test_url)-1-test_url.index(url),url))
    #stdout.write('\r{} done'.format(u))
    #stdout.flush()
    #time.sleep(5)
    #if os.path.isfile(dl_dir+'/{}.docx'):
        #os.rename(dl_dir+'/{}.docx'.format(p.search(url).group(2)),
             #dl_dir+'/{}_{}.docx'.format(company_name,date))
    #else:
        #print(dl_dir+'/{}.docx'.format(p.search(url).group(2))+'stuck')

105